## Etude des sujets des amendements PLFSS - Sasha

## Préparation des données

In [1]:
#!python -m pip install --upgrade pip

In [2]:
#!pip install -q --upgrade pandas  spacy  more_itertools

In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
!python -m spacy download fr_core_news_sm > /dev/null

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python -m pip install --upgrade pip' command.


In [5]:
# Importing modules
import pandas as pd
#amdt = pd.read_csv('https://github.com/leximpact/donnees-extraites-assemblee/blob/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt = pd.read_csv('https://raw.githubusercontent.com/leximpact/donnees-extraites-assemblee/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt.head(5)

,texteLegislatifUid,uid,avantAApres,dispositif,exposeSommaire
0,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000001,A,"I. – Après l’alinéa 13, insérer l’alinéa suiv...",La mise en place d’un accord d’intéressement d...
1,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000002,Apres,À la première phrase du premier alinéa de l’ar...,L’article L 531‑2 du Code de la Sécurité Socia...
2,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000005,A,Compléter l’alinéa 17 par la phrase suivante :...,S’il est louable d’expérimenter des dispositif...
3,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000008,A,"Après l’alinéa 8, insérer les cinq alinéas sui...",Cet amendement permet d’amplifier la portée de...
4,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000010,Apres,Le premier alinéa de l’article L. 521‑1 du cod...,"Pendant plus de cinquante ans, notre politique..."


In [6]:
len(amdt) #Nombre total d'amendements

4797

In [7]:
#On regroupe dans un même texte chaque dispositif et son exposé sommaire
amdt['texte'] = amdt['dispositif'] + amdt['exposeSommaire'] 

## Nettoyage

In [8]:
#pip install nltk

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/sasha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to /Users/sasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
#Tokenisation := découpage du texte en listes de mots
from nltk.tokenize import word_tokenize
tokenized = [ word_tokenize(text) for text in amdt['texte'] ] #return_token

# Normalization

### Retrait des mots de liaison (_stopword_) et de la ponctuation

In [11]:
# #Removing punctuation AND Casing
#( Casing: Est-ce vraiment utile ? Est-ce qu'on ne va pas perdre les Acronymes de vue ?)
new_tokenized = []
for token in tokenized:
    new_tokenized.append([ word.lower() for word in token if word.isalpha()])

In [12]:
print(new_tokenized[4])

['le', 'premier', 'alinéa', 'de', 'l', 'article', 'du', 'code', 'de', 'la', 'sécurité', 'sociale', 'est', 'complété', 'par', 'une', 'phrase', 'ainsi', 'rédigée', 'elles', 'sont', 'universelles', 'pendant', 'plus', 'de', 'cinquante', 'ans', 'notre', 'politique', 'familiale', 'a', 'reposé', 'sur', 'le', 'principe', 'de', 'l', 'universalité', 'cela', 'signifie', 'qu', 'elle', 's', 'adressait', 'à', 'tous', 'les', 'français', 'sans', 'distinction', 'sociale', 'elle', 'reposait', 'sur', 'l', 'idée', 'que', 'chaque', 'enfant', 'à', 'naître', 'est', 'une', 'chance', 'et', 'une', 'richesse', 'pour', 'la', 'france', 'pour', 'son', 'avenir', 'quel', 'que', 'soient', 'les', 'ressources', 'dont', 'disposent', 'les', 'mettre', 'en', 'place', 'ce', 'principe', 'd', 'universalité', 'la', 'politique', 'familiale', 'appelle', 'des', 'outils', 'dits', 'de', 'redistribution', 'horizontale', 'c', 'des', 'mécanismes', 'de', 'solidarité', 'des', 'familles', 'sans', 'enfant', 'envers', 'les', 'familles', 'av

In [13]:
#STOP WORDS
import spacy
#On importe les mots "inutiles" (stopwords) du langage français depuis Space -ET- NLTK
from nltk.corpus import stopwords 
stop_words = stopwords.words("french")

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
for word in fr_stop:
        stop_words.append(word)
del(word)
stop_words = list(stop_words)

#On importe nos propres stopwords depuis le fichier CSV
SW = pd.read_csv('Added_stop_words.csv')
SW = list(SW.StopWords)
final_SW = stop_words+ SW

#On enlève les doublons
df = pd.DataFrame({"StopWords" : final_SW})
final_SW = df.drop_duplicates()
final_SW = list(final_SW.StopWords)
print("NOS MOTS 'INUTILES' : \n \n", final_SW ,'\n')

#STOP WORD REMOVAL
print("AVANT : \n \n", tokenized[4], "\n")
tokenized = []
for token in new_tokenized:
    tokenized.append([ word for word in token if word not in final_SW])
print("APRÈS : \n \n", tokenized[4], "\n")

NOS MOTS 'INUTILES' : 
 
 ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'aura

In [14]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/sasha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
#Lemmatization: on reduit les mots à leur racine
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#amdt_clean= []
#for token in tokenized:
#    amdt_clean.append([lemmatizer.lemmatize(word) for word in token])
#print(amdt_clean[4])

#La lemmatization de nltk est nulle en français. Il faudrait pouvoir réussir à importer celle de Spacy
import spacy
nlp = spacy.load("fr_core_news_md")

#amdt_clean= []
#for token in tokenized:
#    amdt_clean.append([nlp(word) for word in token])
#print("Ci-dessous les mots réduits à leur racine: \n \n", amdt_clean[4])

In [16]:
#Cellule de test
print(lemmatizer.lemmatize('mangée'))
print(lemmatizer.lemmatize('mangées'))
print(lemmatizer.lemmatize('mangé'))
print(lemmatizer.lemmatize('mangés'))
print(lemmatizer.lemmatize('mangez'))
print(lemmatizer.lemmatize('mangeons'))
print(lemmatizer.lemmatize('joliment'))
print(lemmatizer.lemmatize('eaten'))
print(lemmatizer.lemmatize('voiles'))
print(lemmatizer.lemmatize('was'))

mangée
mangées
mangé
mangés
mangez
mangeons
joliment
eaten
voile
wa


In [17]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
stemmer.stem('voudrais')

#Cellule de test
print(stemmer.stem('mangée'))
print(stemmer.stem('mangées'))
print(stemmer.stem('mangé'))
print(stemmer.stem('mangés'))
print(stemmer.stem('mangez'))
print(stemmer.stem('mangeons'))
print(stemmer.stem('joliment'))
print(stemmer.stem('eaten'))
print(stemmer.stem('voiles'))
print(stemmer.stem('was'))

'voudr'

mang
mang
mang
mang
mang
mangeon
jol
eaten
voil
was


In [18]:
#from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger
nlp = spacy.load('fr')
lemmatizer = LefffLemmatizer()
print(lemmatizer.lemmatize('mangée'))
print(lemmatizer.lemmatize('mangées'))
print(lemmatizer.lemmatize('mangé'))
print(lemmatizer.lemmatize('mangés'))
print(lemmatizer.lemmatize('mangez'))
print(lemmatizer.lemmatize('mangeons'))
print(lemmatizer.lemmatize('joliment'))
print(lemmatizer.lemmatize('eaten'))
print(lemmatizer.lemmatize('voiles'))
print(lemmatizer.lemmatize('was'))

ModuleNotFoundError: No module named 'spacy_lefff'

In [ ]:
#Cellule de test
nlp.add_pipe('lemmatizer')
#print(nlp.lemmatizer(u'mangée')) #pk u ?
print(nlp(u'mangées'))
print(nlp('mangé'))
print(nlp('mangés'))
print(nlp('mangez'))
print(nlp('mangeons'))
print(nlp('joliment'))
print(nlp('eaten'))
print(nlp('voiles'))
print(nlp('was'))

In [ ]:
#On enregistre tous les textes en version "clean" pour pouvoir réutiliser cette data plus tard
del(df)
df = pd.DataFrame(data = amdt_clean )
df = df.fillna('') 
print(df.head(10))
df.to_csv('amdements_cleaned.csv')  #Chaque ligne est un amendement

# Les mots les plus cités

In [ ]:
#Counting
from collections import Counter
def most_common_words(amdt_clean, N):
    freq = []
    words = []
    for token in amdt_clean:
        bow = Counter(token)
        freq.append(bow.most_common(N)) 
        #print(bow.most_common(N))
        
        words.append([bow.most_common(N)[i][0] for i in range(N-1)])
    return pd.DataFrame(freq), words

#On ne garde que les N mots les plus utilisés pour chaque amendement
N = 10
freq, words = most_common_words(amdt_clean, N)
#On sauvegarde cette donnée dans un fichier .csv 
#dont les lignes sont les amdts et les colonnes sont les mots et leur fréquence
freq.to_csv('most_common_{nb}_words_per_amdt.csv'.format(nb = N))

print("Ci-dessous un exemple des mots les plus utilisés sur 5 amendements: \n \n ", freq.head(5))


In [ ]:
# Analyse des mots du corpus dans sa globalité
words = pd.DataFrame(words)
mots_uniques = []
for c in words:
    temp = list (words[c].unique())
    for word in temp:
        mots_uniques.append( word )  #List of unique words
        
#On enlève les doublons
mots_uniques = list(dict.fromkeys(mots_uniques))
mots_uniques.sort()
print( len(mots_uniques) )
print(mots_uniques)

# TF IDF
Term frequency - inverse document frequency

In [ ]:
#Mise sous forme 'corpus' (une liste de tous les textes)
corpus = []
for amdt1 in amdt_clean:
    temp = ' '.join(amdt1)
    corpus.append(temp)
    temp = ''
    
#Vectorization - Term Frequency in Global Corpus
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
tf = vectorizer.fit_transform(corpus)

#print(corpus[2:4])

In [ ]:
#Features
feature_names = vectorizer.get_feature_names()
print(feature_names[:10])

In [ ]:
#Génération du corpus index
import re
corpus_index = amdt['uid'].tolist()
print(corpus_index[:5])
#amdt['uid'].nunique() 

In [ ]:
## LAST STOP HERE: POURQUOI A-T-ON UNIQUEMENT DES NAN ?

#IDF - Inverse Document Frequencies
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize and fit TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None)
tf_idf_scores = vectorizer.fit_transform(corpus)
#print(tf_idf_scores)
# get vocabulary of terms
feature_names = vectorizer.get_feature_names()
#print(type(tf_idf_scores.todense()))
print (pd.DataFrame(tf_idf_scores.todense()) )  #ICI on a bien les bonnes valeurs
print(pd.DataFrame(tf_idf_scores.todense()[0][0]))
#print(feature_names)
# create pandas DataFrame with tf-idf scores
df_tf_idf = pd.DataFrame(data = pd.DataFrame(tf_idf_scores.todense()), index=corpus_index, columns=feature_names)
print(df_tf_idf)
#df_tf_idf.fillna(0)
df_tf_idf.replace(np.nan, 0)
#df_tf_idf.reset_index() 
#df_tf_idf.rename_axis('ID') 


In [ ]:
for n in range(5):#range(len(df_tf_idf)):
    #print(n)
    #print(id)
    main = pd.DataFrame()
    main['Most_frequent_words'] = feature_names
    main['Word_occurrence'] = df_tf_idf.iloc[n]  # MAIN ISSUE HERE: Series definition not taking float but convert2 nan
    print(max(df_tf_idf.iloc[n]))
    print(len(df_tf_idf.iloc[n]))
    print(type(df_tf_idf.iloc[n]))
    print(max(main['Word_occurrence']))
    print(type(main['Word_occurrence']))
    print(type(main))
    print(main.head(5))
    main.sort_values(by =['Word_occurrence'], inplace = True, ascending = False, na_position='last')
    print(main.head(5))

In [ ]:
#exclude
#Organizing most common words !Il faut transposer
# tuple_mots = ()
# for n in range(len(df_tf_idf)):
#     print(n)
#     id = df_tf_idf.index[n]
#     print(id)
#     #Creating a DF per amendment
#     main = pd.DataFrame()
#     main['Mots principaux'] = df_tf_idf.columns.T
#     main['Occurence'] = df_tf_idf.iloc[n]
    
    
#     #main['mots'] = df_tf_idf['ID']
    
#     print(max(df_tf_idf.index[n]))
#     #print(main.head(5))
#     #main.sort_values(main[id])

#     #main = main.iloc[:25] #On ne garde que les 25 mots les plus utilisés par amendement
#     #tuple_mots.append(main)

In [ ]:
df_tf_idf.shape

In [ ]:
id = 0

In [ ]:
df_tf_idf.index[0]

In [ ]:
test = df_tf_idf.index[id]